<center><h2>Web Scraping Exercise</h2></center>

In [3]:
# Import packages
import time
import re
import random

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup

import pandas as pd

In [4]:
# Slugify is used to convert his song titles to keys without certain words and special characters
def slugify(s):
    """
    Simplifies ugly strings into something URL-friendly.
    >>> print slugify("[Some] _ Article's Title--")
    some-articles-title
    """

    # "[Some] _ Article's Title--"
    # "[some] _ article's title--"
    s = s.lower()

    # "[some] _ article's_title--"
    # "[some]___article's_title__"
    for c in [' ', '-', '.', '/']:
        s = s.replace(c, '_')

    # "[some]___article's_title__"
    # "some___articles_title__"
    s = re.sub('\W', '', s)

    # "some___articles_title__"
    # "some   articles title  "
    s = s.replace('_', ' ')

    # "some   articles title  "
    # "some articles title "
    s = re.sub('\s+', ' ', s)

    # "some articles title "
    # "some articles title"
    s = s.strip()

    # "some articles title"
    # "some-articles-title"
    s = s.replace(' ', '-')

    return s

Collect all the links for all the songs of interest

In [5]:
# Make a request to the song list page
HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
ALBUM3_URL = "https://www.azlyrics.com/e/edsheeran.html"
album3_resp = ''
while album3_resp == '':
    try:
        album3_resp = requests.get(ALBUM3_URL, headers=HEADERS)
        break
    except:
        print("Connection refused by the server..")
        time.sleep(5)
        continue

In [6]:
# Collect the song lists
album3_soup = BeautifulSoup(album3_resp.text, 'html.parser')
candidate_links = album3_soup.find_all('a')
links = []
for l in candidate_links:
    try:
        if 'edsheeran' in l['href']:
            links.append(re.sub('^\.+', '', l['href']))
    except KeyError:
        pass

Collect the lyrics for each song

In [9]:
songs_list = []
for sl in links[:]:
    LYRICS_URL = "https://www.azlyrics.com%s" % sl
    print(LYRICS_URL)
    # Req/Resp
    song_resp = ''
    while song_resp == '':
        tm = random.choice(range(1200, 5000))/100
        print("Sleeping for %d..." % tm)
        time.sleep(tm)
        try:
            song_resp = requests.get(LYRICS_URL)
            break
        except:
            print("Connection refused by the server..")
            continue
            
    # Get song lyrics
    song_soup = BeautifulSoup(song_resp.text, 'html.parser')
    ringtone_div = song_soup.find('div', attrs={'class' : 'ringtone'})
    lyrics_div = ringtone_div.findNext('div')
    title_div = ringtone_div.findNext('b')
    title = slugify(title_div.text)
    lyrics = re.sub('\n|\r', ' ', lyrics_div.text)
    songs_list.append({'title' : title, 'lyrics' : lyrics})

https://www.azlyrics.com/lyrics/edsheeran/moodyballadofed.html
Sleeping for 27...
https://www.azlyrics.com/lyrics/edsheeran/misery.html
Sleeping for 37...
https://www.azlyrics.com/lyrics/edsheeran/typicalaverage.html
Sleeping for 15...
https://www.azlyrics.com/lyrics/edsheeran/addicted.html
Sleeping for 40...
https://www.azlyrics.com/lyrics/edsheeran/iloveyou.html
Sleeping for 14...
https://www.azlyrics.com/lyrics/edsheeran/openyourears.html
Sleeping for 47...
https://www.azlyrics.com/lyrics/edsheeran/beyondthepale.html
Sleeping for 17...
https://www.azlyrics.com/lyrics/edsheeran/inmemory.html
Sleeping for 33...
https://www.azlyrics.com/lyrics/edsheeran/insomniacslullaby.html
Sleeping for 40...
https://www.azlyrics.com/lyrics/edsheeran/quietballadofed.html
Sleeping for 33...
https://www.azlyrics.com/lyrics/edsheeran/noluck.html
Sleeping for 37...
https://www.azlyrics.com/lyrics/edsheeran/stevensong.html
Sleeping for 12...
https://www.azlyrics.com/lyrics/edsheeran/billyruskin.html
Sleep

https://www.azlyrics.com/lyrics/edsheeran/howwouldyoufeelpaean.html
Sleeping for 43...
https://www.azlyrics.com/lyrics/edsheeran/supermarketflowers.html
Sleeping for 48...
https://www.azlyrics.com/lyrics/edsheeran/barcelona.html
Sleeping for 19...
https://www.azlyrics.com/lyrics/edsheeran/bibiabeyeye.html
Sleeping for 16...
https://www.azlyrics.com/lyrics/edsheeran/nancymulligan.html
Sleeping for 21...
https://www.azlyrics.com/lyrics/edsheeran/savemyself.html
Sleeping for 32...
https://www.azlyrics.com/lyrics/edsheeran/babyonemoretime.html
Sleeping for 28...
https://www.azlyrics.com/lyrics/edsheeran/agsmaoineamhsard.html
Sleeping for 17...
https://www.azlyrics.com/lyrics/edsheeran/bemyhusband.html
Sleeping for 45...
https://www.azlyrics.com/lyrics/edsheeran/candleinthewind2018version.html
Sleeping for 39...
https://www.azlyrics.com/lyrics/edsheeran/castleonthehillacoustic.html
Sleeping for 48...
https://www.azlyrics.com/lyrics/edsheeran/chasingcars.html
Sleeping for 49...
https://www.a

In [10]:
# Write to DF
df = pd.DataFrame(songs_list)
df.to_csv('ed-sheeran-2.csv', sep='|', index=False)

In [11]:
print(df.shape)

(167, 2)
